# Oanda: How to create a stop loss & take profit

In [2]:
from trader import ConTrader

In [4]:
trader = ConTrader("Oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [6]:
trader.get_most_recent()

In [8]:
trader.define_strategy()

In [10]:
current_price = trader.data[trader.instrument].iloc[-1]
current_price

1.13677

In [12]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                           sl_distance = 0.01)
order

{'id': '398',
 'time': '2025-04-30T02:41:25.998442555Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '398',
 'requestID': '79380768650058931',
 'type': 'MARKET_ORDER',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'stopLossOnFill': {'distance': '0.01', 'timeInForce': 'GTC'}}

In [14]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '401',
 'time': '2025-04-30T02:41:26.727628010Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '400',
 'requestID': '115409565671389651',
 'type': 'ORDER_FILL',
 'orderID': '400',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13661,
 'fullVWAP': 1.13661,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13661, 'liquidity': '500000'},
   {'price': 1.1366, 'liquidity': '2500000'},
   {'price': 1.13659, 'liquidity': '2000000'},
   {'price': 1.13658, 'liquidity': '5000000'},
   {'price': 1.13655, 'liquidity': '10000000'},
   {'price': 1.13652, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13678, 'liquidity': '500000'},
   {'price': 1.13679, 'liquidity': '500000'},
   {'price': 1.1368, 'liquidity': '2000000'},
   {'price': 1.13681, 'liquidity': '2000000'},
   {'price': 1.13682, 'liquidity': '5000000'},
   {'price': 1.13684, 'liquidity': '10000000'},
   {'pr

# Adding Limit Orders to Trading Bot (Take profit & Stop Loss)

### Oanda: Contrader Class without Trake Profits & Stop Loss

In [18]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time

In [20]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # new (Python 3.12)
                break
            
    def start_trading(self, days, max_attempts = 5, wait = 20, wait_increase = 0): # NEW, erroe handling
        attempt = 0
        success = False 
        while True:
            try:
                self.get_most_recent(days)
                self.stream_data(self.instrument)
            except Exception as e:
                print(e, end = " | ")
            else:
                success = True
                break
            finally:
                attempt += 1
                print("Attempt: {}".format(attempt), end = 'n/')
                if success == False:
                    if attempt >= max_attempts:
                        print("Max_attempts reached!")
                        try: # for termination 
                            time.sleep(wait)
                            self.terminate_session(cause = "Unexpected Session Stop (too many errors).")
                        except Exception as e:
                            print(e, end = " | ")
                            print("Could not terminate the session.")
                        finally:
                            break
                    else:
                        time.sleep(wait)
                        wait += wait_increase
                        self.tick_data = pd.DataFrame()
                     
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [22]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [24]:
trader.start_trading(days = 5, max_attempts = 3, wait = 20, wait_increase = 0)

1
----------------------------------------------------------------------------------------------------
2025-04-30T02:41:38.152715523Z | GOING LONG
2025-04-30T02:41:38.152715523Z | units = 100.0 | price = 1.13673 | P&L = 0.0 | Cum P&L = 0.0
----------------------------------------------------------------------------------------------------

100
----------------------------------------------------------------------------------------------------
2025-04-30T02:43:39.091574926Z | GOING NEUTRAL
2025-04-30T02:43:39.091574926Z | units = -100.0 | price = 1.1364 | P&L = 9.881 | Cum P&L = 9.881
----------------------------------------------------------------------------------------------------

Scheduled Session End.
Attempt: 1n/

In [26]:
trader.data

,EUR_USD,returns,position
2025-04-25 02:41:00+00:00,1.134600,NaN,NaN
2025-04-25 02:42:00+00:00,1.134620,0.000018,-1.0
2025-04-25 02:43:00+00:00,1.134460,-0.000141,1.0
2025-04-25 02:44:00+00:00,1.134480,0.000018,-1.0
2025-04-25 02:45:00+00:00,1.134520,0.000035,-1.0
...,...,...,...
2025-04-30 02:38:00+00:00,1.136780,0.000018,-1.0
2025-04-30 02:39:00+00:00,1.136790,0.000009,-1.0
2025-04-30 02:40:00+00:00,1.136770,-0.000018,1.0
2025-04-30 02:42:00+00:00,1.136615,-0.000136,1.0


In [28]:
trader.position

0

## Stop loss and take profit

In [31]:
trader

In [33]:
current_price = trader.data[trader.instrument].iloc[-1]
current_price

1.1364049999999999

In [39]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True, # open a long position
                           sl_distance = 0.01)
order

{'id': '410',
 'time': '2025-04-30T02:49:38.428327175Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '410',
 'requestID': '97395169228583757',
 'type': 'MARKET_ORDER',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'stopLossOnFill': {'distance': '0.01', 'timeInForce': 'GTC'}}

In [41]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True) # Closing position manually
order

{'id': '413',
 'time': '2025-04-30T02:50:45.988826293Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '412',
 'requestID': '97395169509646222',
 'type': 'ORDER_FILL',
 'orderID': '412',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13665,
 'fullVWAP': 1.13665,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13665, 'liquidity': '500000'},
   {'price': 1.13664, 'liquidity': '2500000'},
   {'price': 1.13663, 'liquidity': '2000000'},
   {'price': 1.13662, 'liquidity': '5000000'},
   {'price': 1.13659, 'liquidity': '10000000'},
   {'price': 1.13656, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13681, 'liquidity': '500000'},
   {'price': 1.13682, 'liquidity': '500000'},
   {'price': 1.13683, 'liquidity': '2000000'},
   {'price': 1.13684, 'liquidity': '2000000'},
   {'price': 1.13685, 'liquidity': '5000000'},
   {'price': 1.13687, 'liquidity': '10000000'},
   {'p

#### Take Profit

In [63]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            tp_price = 1.14) # Adjust price as needed.
order

{'id': '427',
 'time': '2025-04-30T03:09:16.672470708Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '427',
 'requestID': '79380775660153170',
 'type': 'MARKET_ORDER_REJECT',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'takeProfitOnFill': {'price': 1.13999999999999, 'timeInForce': 'GTC'},
 'rejectReason': 'TAKE_PROFIT_ON_FILL_PRICE_PRECISION_EXCEEDED'}

In [65]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '429',
 'time': '2025-04-30T03:09:46.416153574Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '428',
 'requestID': '97395174291986829',
 'type': 'ORDER_FILL',
 'orderID': '428',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13735,
 'fullVWAP': 1.13735,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13735, 'liquidity': '500000'},
   {'price': 1.13734, 'liquidity': '2500000'},
   {'price': 1.13733, 'liquidity': '2000000'},
   {'price': 1.13732, 'liquidity': '5000000'},
   {'price': 1.13729, 'liquidity': '10000000'},
   {'price': 1.13726, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13749, 'liquidity': '500000'},
   {'price': 1.1375, 'liquidity': '500000'},
   {'price': 1.13751, 'liquidity': '2000000'},
   {'price': 1.13752, 'liquidity': '2000000'},
   {'price': 1.13753, 'liquidity': '5000000'},
   {'price': 1.13755, 'liquidity': '10000000'},
   {'pr

#### Combining SL and TP (Stop Loss and Take Profit)

In [67]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            sl_distance = 0.01, tp_price = 1.11)
order

{'id': '431',
 'time': '2025-04-30T03:10:37.368561760Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '430',
 'requestID': '61366377490745039',
 'type': 'ORDER_FILL',
 'orderID': '430',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13744,
 'fullVWAP': 1.13744,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13728, 'liquidity': '500000'},
   {'price': 1.13727, 'liquidity': '2500000'},
   {'price': 1.13726, 'liquidity': '2000000'},
   {'price': 1.13725, 'liquidity': '5000000'},
   {'price': 1.13722, 'liquidity': '10000000'},
   {'price': 1.13719, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '500000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '2000000'},
   {'price': 1.13748, 'liquidity': '5000000'},
   {'price': 1.1375, 'liquidity': '10000000'},
   {'pri

In [69]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '433',
 'time': '2025-04-30T03:11:59.480379934Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '432',
 'requestID': '61366377834737205',
 'type': 'ORDER_FILL',
 'orderID': '432',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13748,
 'fullVWAP': 1.13748,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13748, 'liquidity': '500000'},
   {'price': 1.13747, 'liquidity': '2500000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13745, 'liquidity': '5000000'},
   {'price': 1.13742, 'liquidity': '10000000'},
   {'price': 1.13739, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13762, 'liquidity': '500000'},
   {'price': 1.13763, 'liquidity': '500000'},
   {'price': 1.13764, 'liquidity': '2000000'},
   {'price': 1.13765, 'liquidity': '2000000'},
   {'price': 1.13766, 'liquidity': '5000000'},
   {'price': 1.13768, 'liquidity': '10000000'},
   {'p

#### Trailing Stop Loss

Trails the price takes a stop loss based on tsl_distance

In [72]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            tsl_distance = 0.01)
order

{'id': '435',
 'time': '2025-04-30T03:16:06.561249891Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '434',
 'requestID': '61366378866699789',
 'type': 'ORDER_FILL',
 'orderID': '434',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13763,
 'fullVWAP': 1.13763,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13748, 'liquidity': '500000'},
   {'price': 1.13747, 'liquidity': '2500000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13745, 'liquidity': '5000000'},
   {'price': 1.13742, 'liquidity': '10000000'},
   {'price': 1.13739, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13763, 'liquidity': '500000'},
   {'price': 1.13764, 'liquidity': '500000'},
   {'price': 1.13765, 'liquidity': '2000000'},
   {'price': 1.13766, 'liquidity': '2000000'},
   {'price': 1.13767, 'liquidity': '5000000'},
   {'price': 1.13769, 'liquidity': '10000000'},
   {'pr

In [77]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '437',
 'time': '2025-04-30T03:19:28.823027293Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '436',
 'requestID': '79380778227569197',
 'type': 'ORDER_FILL',
 'orderID': '436',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13776,
 'fullVWAP': 1.13776,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13776, 'liquidity': '500000'},
   {'price': 1.13775, 'liquidity': '2500000'},
   {'price': 1.13774, 'liquidity': '2000000'},
   {'price': 1.13773, 'liquidity': '5000000'},
   {'price': 1.1377, 'liquidity': '10000000'},
   {'price': 1.13767, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13791, 'liquidity': '500000'},
   {'price': 1.13792, 'liquidity': '500000'},
   {'price': 1.13793, 'liquidity': '2000000'},
   {'price': 1.13794, 'liquidity': '2000000'},
   {'price': 1.13795, 'liquidity': '5000000'},
   {'price': 1.13797, 'liquidity': '10000000'},
   {'pr